In [2]:
#Import Dependencies
import pandas as pd

In [19]:
# Import the world internet data from world bank and rename columns for merging with location data
world_data= pd.read_csv("../Resources/data-raw/Data_Extract_From_World_Development_Indicators/world_bank_internet_data.csv", skiprows=[2395,2396,2397,2398,2399,2400])
world_data.rename(columns={"Country Name": "Country", "Country Code": "Codes"}, inplace=True)
world_data

,Country,Codes,Series Name,Series Code,1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,0,0,0,0,0,..,...,5,5.454545455,5.9,7,8.26,11,13.5,16.8,17.6,18.4
1,Afghanistan,AFG,Secure Internet servers,IT.NET.SECR,..,..,..,..,..,..,...,16,38,46,57,74,426,1577,1965,1047,1362
2,Afghanistan,AFG,Secure Internet servers (per 1 million people),IT.NET.SECR.P6,..,..,..,..,..,..,...,0.531254164,1.219458267,1.425490598,1.708079913,2.15031248,12.03967054,43.44818099,52.86248045,27.5223881,34.98736306
3,Afghanistan,AFG,Mobile cellular subscriptions,IT.CEL.SETS,0,0,0,0,0,0,...,13797879,15340115,16807156,18407168,19709038,21602982,23929713,21976355,22580071,22678024
4,Afghanistan,AFG,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,0,0,0,0,0,0,...,45.81362616,49.22797697,52.08357627,55.15951463,57.27106807,61.05463771,65.92913406,59.12084823,59.3560197,58.25581184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2389,Africa Western and Central,AFW,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,0,0.000409304,0.005632347,0.009679299,0.012663403,0.026080925,...,59.25287866,68.29760995,76.34620664,81.72986151,86.70769269,86.32538724,84.6725609,94.21880758,94.63652266,99.61737975
2390,Africa Western and Central,AFW,Fixed broadband subscriptions,IT.NET.BBND,..,..,..,..,..,..,...,..,400964,399466,489777,565867,971317,980402,1084105,1209936,1743931
2391,Africa Western and Central,AFW,Fixed broadband subscriptions (per 100 people),IT.NET.BBND.P2,..,..,..,..,..,..,...,..,0.110297236,0.108219429,0.135123016,0.145258497,0.242682761,0.238441559,0.274406197,0.278968433,0.391018789
2392,Africa Western and Central,AFW,Fixed telephone subscriptions,IT.MLT.MAIN,692452,764375,823937,899859,1001339,1092484,...,3164968,2941315,2925394,3055563,2943790,2871319,2652247,2531706,2431126,2612210


In [48]:
#Import the coordinate csv file, rename columns for merging with each other later on
coordinates = pd.read_csv("../Resources/data-raw/countries/world_country.csv")
coordinates.rename(columns={"iso_con" : "Codes", "lat": "Latitude", "lon": "Longitude"},inplace=True)
coordinates.drop(columns=["Unnamed: 0", "country"],inplace=True)
coordinates

,Longitude,Latitude,Codes
0,66.238514,33.768006,AFG
1,19.999962,41.000028,ALB
2,2.999983,28.000027,DZA
3,-170.692511,-14.289304,ASM
4,1.573203,42.540717,AND
...,...,...,...
240,-176.204224,-13.289402,WLF
241,-13.892143,24.168196,ESH
242,47.891527,16.347124,YEM
243,27.558988,-14.518912,ZMB


In [49]:
final_world = pd.merge(world_data, coordinates, on="Codes", how="left")
final_world

,Country,Codes,Series Name,Series Code,1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],Longitude,Latitude
0,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,0,0,0,0,0,..,...,5.9,7,8.26,11,13.5,16.8,17.6,18.4,66.238514,33.768006
1,Afghanistan,AFG,Secure Internet servers,IT.NET.SECR,..,..,..,..,..,..,...,46,57,74,426,1577,1965,1047,1362,66.238514,33.768006
2,Afghanistan,AFG,Secure Internet servers (per 1 million people),IT.NET.SECR.P6,..,..,..,..,..,..,...,1.425490598,1.708079913,2.15031248,12.03967054,43.44818099,52.86248045,27.5223881,34.98736306,66.238514,33.768006
3,Afghanistan,AFG,Mobile cellular subscriptions,IT.CEL.SETS,0,0,0,0,0,0,...,16807156,18407168,19709038,21602982,23929713,21976355,22580071,22678024,66.238514,33.768006
4,Afghanistan,AFG,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,0,0,0,0,0,0,...,52.08357627,55.15951463,57.27106807,61.05463771,65.92913406,59.12084823,59.3560197,58.25581184,66.238514,33.768006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,Africa Western and Central,AFW,Mobile cellular subscriptions (per 100 people),IT.CEL.SETS.P2,0,0.000409304,0.005632347,0.009679299,0.012663403,0.026080925,...,76.34620664,81.72986151,86.70769269,86.32538724,84.6725609,94.21880758,94.63652266,99.61737975,NaN,NaN
2435,Africa Western and Central,AFW,Fixed broadband subscriptions,IT.NET.BBND,..,..,..,..,..,..,...,399466,489777,565867,971317,980402,1084105,1209936,1743931,NaN,NaN
2436,Africa Western and Central,AFW,Fixed broadband subscriptions (per 100 people),IT.NET.BBND.P2,..,..,..,..,..,..,...,0.108219429,0.135123016,0.145258497,0.242682761,0.238441559,0.274406197,0.278968433,0.391018789,NaN,NaN
2437,Africa Western and Central,AFW,Fixed telephone subscriptions,IT.MLT.MAIN,692452,764375,823937,899859,1001339,1092484,...,2925394,3055563,2943790,2871319,2652247,2531706,2431126,2612210,NaN,NaN


In [50]:
df = final_world.loc[final_world.isna().any(axis=1)]
df.Country.unique()

array(['Channel Islands', 'Curacao', 'Kosovo', 'Micronesia, Fed. Sts.',
       'Namibia', 'Niger', 'Africa Eastern and Southern', 'Arab World',
       'Central Europe and the Baltics', 'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
 